In [1]:
from pyspark.sql import SparkSession
# Inicializa uma sessão do Spark com as configurações do Iceberg

spark = SparkSession.builder \
  .appName("IcebergLocalDevelopment") \
  .config("spark.driver.host", "localhost") \
  .config('spark.jars.packages', 'org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.6.1') \
  .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
  .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog") \
  .config("spark.sql.catalog.local.type", "hadoop") \
  .config("spark.sql.catalog.local.warehouse", "spark-warehouse/iceberg") \
  .config("spark.sql.catalog.local.create-namespace", "true") \
  .config("spark.driver.bindAddress", "localhost") \
  .getOrCreate()

25/04/21 23:16:32 WARN Utils: Your hostname, gabriel-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
25/04/21 23:16:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/gabriel/Documents/GitHub/eg-dados-apache-spark/.venv/lib/python3.13/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/gabriel/.ivy2/cache
The jars for the packages stored in: /home/gabriel/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9f189d52-97b7-47fc-95d5-e54e9f27e767;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.6.1 in central
:: resolution report :: resolve 153ms :: artifacts dl 4ms
	:: modules in use:
	org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.6.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.

In [2]:
# Criação da tabela
spark.sql(
    """
    CREATE TABLE IF NOT EXISTS local.iceberg.covid_daily_vaccinations (
        date DATE,
        location STRING,
        daily_vaccinations DOUBLE,
        people_fully_vaccinated DOUBLE
    ) USING iceberg PARTITIONED BY (year(date));
    """
)

DataFrame[]

In [3]:
spark.sql("SELECT * FROM local.iceberg.covid_daily_vaccinations").show()

25/04/21 23:16:44 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


+----------+-----------+------------------+-----------------------+
|      date|   location|daily_vaccinations|people_fully_vaccinated|
+----------+-----------+------------------+-----------------------+
|2022-01-01|Afghanistan|           16935.0|                   NULL|
|2022-01-02|Afghanistan|           17269.0|                   NULL|
|2022-01-03|Afghanistan|           17604.0|                   NULL|
|2022-01-04|Afghanistan|           17604.0|                   NULL|
|2022-01-05|Afghanistan|           17604.0|                   NULL|
|2022-01-06|Afghanistan|           17604.0|                   NULL|
|2022-01-07|Afghanistan|           17604.0|                   NULL|
|2022-01-08|Afghanistan|           17604.0|                   NULL|
|2022-01-09|Afghanistan|           17604.0|              3809353.0|
|2022-01-10|Afghanistan|           16642.0|                   NULL|
|2022-01-11|Afghanistan|           15680.0|              3814837.0|
|2022-01-12|Afghanistan|           15287.0|     

In [4]:
from pyspark.sql.functions import to_date, col

df = spark.read.csv("data/covid-19/vaccinations.csv", header=True)

df = df.withColumn("date", to_date(df["date"], "yyyy-MM-dd"))
df = df.withColumn("daily_vaccinations", col("daily_vaccinations").cast("double"))
df = df.withColumn("people_fully_vaccinated", col("people_fully_vaccinated").cast("double"))

df.writeTo("local.iceberg.covid_daily_vaccinations").append()

In [5]:
spark.sql(
    """
    INSERT INTO local.iceberg.covid_daily_vaccinations
    VALUES
        (DATE('2025-04-21'), 'Brazil', 20000.0, 15000000.0),
        (DATE('2025-04-20'), 'USA', 30000.0, 25000000.0),
        (DATE('2025-04-19'), 'India', 40000.0, 35000000.0)
    """
)

DataFrame[]

In [6]:
spark.sql(
    """ 
    UPDATE local.iceberg.covid_daily_vaccinations
    SET daily_vaccinations = 50000.0
    WHERE location = 'Brazil'
    AND date = DATE('2025-04-21')
    """
)


DataFrame[]

In [ ]:
spark.sql(
    """
    SELECT * FROM local.iceberg.covid_daily_vaccinations 
    WHERE location = 'Brazil' AND date = DATE('2025-04-21')
    """
).show()

#
# A fatal error has been detected by the Java Runtime Environment:
#
#  SIGSEGV (0xb) at pc=0x000072290532ed38, pid=23711, tid=24444
#
# JRE version: OpenJDK Runtime Environment (21.0.6+7) (build 21.0.6+7-Ubuntu-124.04.1)
# Java VM: OpenJDK 64-Bit Server VM (21.0.6+7-Ubuntu-124.04.1, mixed mode, sharing, tiered, compressed oops, compressed class ptrs, g1 gc, linux-amd64)
# Problematic frame:
# V  [libjvm.so+0xf2ed38]  SymbolTable::do_lookup(char const*, int, unsigned long)+0xd8
#
# Core dump will be written. Default location: Core dumps may be processed with "/usr/share/apport/apport -p%p -s%s -c%c -d%d -P%P -u%u -g%g -- %E" (or dumping to /home/gabriel/Documents/GitHub/eg-dados-apache-spark/core.23711)
#
# An error report file with more information is saved as:
# /home/gabriel/Documents/GitHub/eg-dados-apache-spark/hs_err_pid23711.log
[705.896s][warning][os] Loading hsdis library failed
#
# If you would like to submit a bug report, please visit:
#   https://bugs.launchpad.net/ubuntu

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/gabriel/Documents/GitHub/eg-dados-apache-spark/.venv/lib/python3.13/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ~~~~~~~~~~~~~~~~~~~~^^
  File "/home/gabriel/.local/share/uv/python/cpython-3.13.3-linux-x86_64-gnu/lib/python3.13/socket.py", line 719, in readinto
    return self._sock.recv_into(b)
           ~~~~~~~~~~~~~~~~~~~~^^^
ConnectionResetError: [Errno 104] Connection reset by peer

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/gabriel/Documents/GitHub/eg-dados-apache-spark/.venv/lib/python3.13/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/gabriel/Documents/GitHub/eg-dados-apache-spark/.venv/lib/python3.13/site-packages/py4j/clientserver.py",

Py4JError: An error occurred while calling o88.showString